In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skimage.exposure as exp
import glob
import scipy.io as scio
import cv2
import os

In [ ]:
def get_position(image, filter_width, filter_height):
    
    """
    This function automatically detects the position of our 3D-model in the tank from an 'image'.
    
    Each image is filtered with a filter_width*filter_height sized filter, allowing automatic detection of the
    model's position while distinguishing it from wave breaking, which typically appears as a less uniform
    bright white pattern on the images.
    
    """
    
    # Define the size of your filter (will vary if we track smaller models than Default)
    image = exp.adjust_log(exp.adjust_gamma(image,2),1)
    intensity = (image[:,:,0] + image[:,:,1]+image[:,:,2])
    
    # Calculate the shape of the result image
    result_shape = (intensity.shape[0] - filter_height + 1, intensity.shape[1] - filter_width + 1)

    # Extract all possible ROIs at once using NumPy slicing
    rois = np.lib.stride_tricks.sliding_window_view(intensity, (filter_height, filter_width))

    # Compute the mean intensity for all ROIs at once
    result = np.mean(rois, axis=(2, 3))

    # Find the contour with the highest intensity
    max_intensity_contour_idx = np.argmax(result)
    
    # Convert the 1D index to 2D (x, y) coordinates
    y, x = np.unravel_index(max_intensity_contour_idx, result_shape)

    # Calculate the (x, y) coordinates in the original image
    x_original = x + filter_width / 2  # Add half of the filter width
    y_original = y + filter_height / 2  # Add half of the filter height
    
    return x_original,y_original

In [ ]:
def track_per_ws(wind_speed, model, reflects,  fw, fh, nruns=5):
    
    """
    For the wind speed and model specified in entry of this function, for all 5 runs of the experiment,
    we track the position of the model in the flume using the get_position() function and save it to a .mat file.

    At low wind speeds (wind speed at 70cm height <= 2 m/s), the water's flat surface exhibits high 
    reflectivity, causing it to reflect objects within the flume, such as lightbulbs. Then, if the
    'reflects' parameter is set to 'True,' we apply an additional mask to the images, 
    specifically targeting areas where reflections of objects outside our area of interest may appear.
    
    In order to remove the remaining errors of the automatic detection, we compute the gradient between successive
    positions over time, and remove unrealistic gradients.
    
    """
    
    print('Wind speed: '+str(wind_speed)+', Model: '+model+' in process....')
    
    path_write = '/home/nbourg/Bureau/clean_plots_treatment_bb/process_scripts/Results_Traj_Detection/Raw/'
    path_to_file = path_write+'WS'+str(wind_speed)+'Model'+model+'.mat'
    
    xcoord = [[]]*nruns
    ycoord = [[]]*nruns
    
    # Iterate over all runs
    for runs in range(1,nruns+1):
        ws = str(wind_speed)
        run = str(runs)

        path = '/home/nbourg/Manip_BB_clean/Manip/wind_'+ws+'ms/Run'+run+'/'+model+'/'
        
        # check if we have data
        if os.path.exists(path):
            len_files = len(os.listdir(path))

            if len_files > 2:

                path_im = np.sort(glob.glob(path+'*.jpg'))

                x_pos  = []
                y_pos = []

                for i in range(len(path_im)):
                    image = cv2.imread(path_im[i])

                    # Define the coordinates of the triangle vertices to remove the parts of the images that are not
                    # water (e.g. wave flume walls)
                    triangle_vertices = np.array([[0, 806], [641, 0], [0, 0]])
                    triangle_vertices2 = np.array([[1920, 806], [1920-641, 0], [1920, 0]])

                    # Create a mask
                    mask = np.zeros(image.shape[:2], dtype=np.uint8)
                    cv2.fillPoly(mask, [triangle_vertices,triangle_vertices2], 255)

                    # Convert image to float and set masked region to NaN
                    image = image.astype(float)
                    image[mask == 255,0] = 0
                    image[mask == 255,1] = 0
                    image[mask == 255,2] = 0

                    # Do same thing if you are at low wind speeds
                    if reflects:
                        poly1 = np.array([[920,410], [977,410], [977,0], [920,0]])
                        poly2 = np.array([[1167,19], [1279,19], [1279,117], [1167,117]])
                        poly3 = np.array([[80,850], [135,947], [190,840]])

                        # Create a mask
                        mask_reflects = np.zeros(image.shape[:2], dtype=np.uint8)
                        cv2.fillPoly(mask_reflects, [poly1,poly2,poly3], 255)

                        image[mask_reflects == 255,0] = 0
                        image[mask_reflects == 255,1] = 0
                        image[mask_reflects == 255,2] = 0

                    x,y = get_position(image,fw,fh)
                    x_pos.append(x)
                    y_pos.append(y)

                t1 = np.array(x_pos)
                t2 = np.array(y_pos)
                
                # Compute gradient betwenn positions to remove weird stuff 
                diff_x = np.gradient(np.gradient(t1))
                diff_y = np.gradient(np.gradient(t2))

                condi = np.logical_or(np.abs(diff_x)>40,np.abs(diff_y)>40)
                t1[condi] = np.nan
                t2[condi] = np.nan

                xcoord[runs-1] = t1
                ycoord[runs-1] = t2
                
    # Save trajectories to a .mat file
    if len(xcoord)>0:
        dicpos = {"x": xcoord, "y": ycoord}
        scio.savemat(path_to_file, dicpos)

    return xcoord, ycoord

In [ ]:
def apply_calib(x, y, wind_speed, model, nruns=5):
    
    path_write = '/home/nbourg/Bureau/clean_plots_treatment_bb/process_scripts/Results_Traj_Detection/Calib/'
    path_to_file = path_write+'WS'+str(wind_speed)+'Model'+model+'_Calib.mat'
    
    x_calib = [[]]*nruns
    y_calib = [[]]*nruns
    
    # Iterate over all runs
    for run in range(1,nruns+1):
        
        path = '/home/nbourg/Manip_BB_clean/Manip/wind_'+str(wind_speed)+'ms/Run'+str(run)+'/'+model+'/'
        
        # check if we have data
        if os.path.exists(path):
            len_files = len(os.listdir(path))

            if len_files > 2:

                xcoord = x[run-1]
                ycoord = y[run-1]

                filename_grid = '/home/nbourg/Bureau/clean_plots_treatment_bb/process_scripts/data_coord_grid_matrix_camera2.mat'

                fgrid = scio.loadmat(filename_grid)

                # Get calibration grid points we computed beforehand
                grid_x = fgrid['Final_gridpoints_order_x']
                grid_y = fgrid['Final_gridpoints_order_y']

                sx, sy = grid_x.shape

                # Initialize matrices for distance calculations
                dist_x = np.nan*np.ones((sx,sy))
                dist_y = np.nan*np.ones((sx,sy))

                # Fill distance matrices with coordinates
                for i in range(sx):
                    dist_y[i,:] = np.arange(0, 20*(sy), step=20)
                for i in range(sy):
                    dist_x[:,i] = np.arange(0,20*(sx), step=20)

                xcoord_dist = np.nan*np.ones(xcoord.shape)
                ycoord_dist = np.nan*np.ones(ycoord.shape)

                # Iterate through trajectory points
                for i in range(len(xcoord)):

                    # Calculate distances between trajectory point and grid points
                    diff = np.sqrt(np.power(xcoord[i]-grid_x,2) + np.power(ycoord[i]-grid_y,2))
                    min_val = np.min(diff)

                    # find closest grid points to detected trajectory point
                    row,col = np.unravel_index(diff.argmin(), diff.shape)

                    if row > 12-1 or col > 28-1:
                        a=2
                    else:
                        delta_grid_x = np.abs(grid_x[row,col] - grid_x[row+1,col])
                        delta_grid_y = np.abs(grid_y[row,col] - grid_y[row,col+1])

                        # Get distance between the closest grid point and the detected trajectory point
                        dist_from_cr_x = np.abs(xcoord[i] - dist_x[row,col])/delta_grid_x
                        dist_from_cr_y = np.abs(ycoord[i] - dist_y[row,col])/delta_grid_y
                        
                        # the delta_x and delta_y of the wooden grid used to calibrate measure 20cm
                        delta_x = 20
                        delta_y = 20 
                        
                        # Get (x,y) positions after calibration
                        if grid_y[row,col] > ycoord[i]:
                            ycoord_dist[i] = dist_y[row,col] - dist_from_cr_y*delta_y 
                        else:
                            ycoord_dist[i] = dist_y[row,col] + dist_from_cr_y*delta_y

                        if grid_x[row,col] > xcoord[i]:
                            xcoord_dist[i] = dist_x[row,col] - dist_from_cr_x*delta_x
                        else:
                            xcoord_dist[i] = dist_x[row,col] + dist_from_cr_x*delta_x

                x_calib[run-1] = xcoord_dist
                y_calib[run-1] = ycoord_dist
    
    # Save calibrated trajectories to .mat file
    if len(x_calib)>0:
        dicpos = {"x_calib": x_calib, "y_calib": y_calib}
        scio.savemat(path_to_file, dicpos)

        print('Wind speed: '+str(wind_speed)+', Model: '+model+' done!')


In [ ]:
models = ['L11111',
         'R11111',
         'L11211',
         'R11211',
         'L12111',
         'R12111',
         'L12211',
         'R12211',
         'L21111',
         'R21111',
         'LC1111',
         'RC1111',
         'LL1111',
         'RL1111',
         'L11111_7tent',
         'L11111_longtent',
          'R11111_longtent',
         'L11111_notent',
         'R11111_notent']

for wind_speed,reflects in zip([1,2,4,6,7,8,10,12], [True,True,False,False,False,False,False,False]):
    for model in models:
        if model=='L21111' or model =='R21111':
            fw = fh = 7
        else:
            fw = fh = 10
        x, y = track_per_ws(wind_speed, model, reflects = reflects,fw=fw,fh=fh)
        apply_calib(x, y, wind_speed, model)